In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
sys.path.append('../')
import utils

In [3]:
v_data, e_data, v_sets, e_sets, core_targets, ext_targets, core_testing = utils.load_for_jupyter()

Dataset already downloaded. Loading it from file system
LOADING DATA: 0.99 s
PREPROCESSING: 1.06 s


In [4]:
v_data_new = v_data.drop(['CoreCaseGraphID', 'ExtendedCaseGraphID', 'testingFlag'], axis=1)

In [5]:
core_targets_new = core_targets.rename(columns={'CaseID': 'CoreCaseGraphID'})
core_targets_new.index.name = 'node_id'

ext_targets_new = ext_targets.rename(columns={'CaseID': 'ExtendedCaseGraphID'})
ext_targets_new.index.name = 'node_id'

core_testing_new = core_testing.rename(columns={'CaseID': 'CoreCaseGraphID'})
core_testing_new.index.name = 'node_id'

In [6]:
v_data_new = pd.merge(v_data_new, core_targets_new, left_index=True, right_index=True, how='left')
v_data_new = pd.merge(v_data_new, ext_targets_new, left_index=True, right_index=True, how='left')

In [7]:
# For some reason some nodes have 2 different core case IDs... filtering to just take the first
for ind, row in v_data.iterrows():
    v_data_new['CoreCaseGraphID'][ind] = core_testing.loc[core_testing.index == ind].CaseID.values[0] if len(core_testing.loc[core_testing.index == ind].CaseID.values) != 0 else row['CoreCaseGraphID']

In [8]:
tsf = pd.DataFrame(v_data_new.index)
tsf = tsf.set_index('node_id')
tsf['testingFlag'] = np.NaN
for ind, row in tsf.iterrows():
    tsf['testingFlag'][ind] = 0 if len(core_targets.loc[core_targets.index == ind]) != 0 or len(ext_targets.loc[ext_targets.index == ind]) != 0 else row.testingFlag
for ind, row in tsf.iterrows():
    tsf['testingFlag'][ind] = 1 if len(core_testing.loc[core_testing.index == ind]) != 0 else row.testingFlag
v_data_new = pd.merge(v_data_new, tsf, left_index=True, right_index=True, how='left')

In [9]:
v_data_new

,Label,Revenue Size Flag,Account ID String,Address,Person or Organisation,Name,Income Size Flag,CoreCaseGraphID,ExtendedCaseGraphID,testingFlag
node_id,,,,,,,,,,
100100,Customer,NaN,NaN,NaN,Person,CHURCH STREET TRUSTEES LIMITED as Trustees for...,medium,NaN,NaN,NaN
100101,Customer,NaN,NaN,NaN,Person,CN BETA LIMITED,high,NaN,NaN,NaN
100102,Customer,NaN,NaN,NaN,Organisation,CHURCH STREET TRUSTEES LIMITED AS TRUSTEES OF ...,high,NaN,NaN,NaN
100103,Customer,NaN,NaN,NaN,Organisation,BOS SECRETARIES ((JERSEY) LIMITED (formerly IN...,medium,NaN,NaN,NaN
100104,Customer,NaN,NaN,NaN,Person,CHURCH STREET TRUSTEES LIMITED,high,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
20030025114,Derived Entity,NaN,NaN,NaN,Person,gMsblvnPVn,NaN,NaN,NaN,NaN
20030025115,Derived Entity,NaN,NaN,NaN,Person,aWHuteBUnk,NaN,NaN,NaN,NaN
20030025116,Derived Entity,NaN,NaN,NaN,Person,E9VrY0MBsT,NaN,NaN,2031.0,0.0


In [10]:
v_data_new.testingFlag.value_counts()

0.0    37801
1.0     2660
Name: testingFlag, dtype: int64

In [11]:
v_data_new.CoreCaseGraphID.value_counts()

1745.0    12
3480.0     7
1731.0     7
1685.0     7
2617.0     7
          ..
491.0      1
2440.0     1
514.0      1
806.0      1
2542.0     1
Name: CoreCaseGraphID, Length: 4000, dtype: int64

In [12]:
v_data_new.ExtendedCaseGraphID.value_counts()

333.0     79
421.0     68
289.0     64
787.0     61
1140.0    56
          ..
2126.0     1
2252.0     1
3760.0     1
279.0      1
3367.0     1
Name: ExtendedCaseGraphID, Length: 3013, dtype: int64

In [ ]:
v_data_new[~v_data_new.index.duplicated(keep='first')]